In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

# ==============================
# SETTINGS
# ==============================
DATASET_PATH = "/kaggle/input/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration"
IMG_SIZE = 300
BATCH_SIZE = 16
EPOCHS_HEAD = 15
EPOCHS_FINE = 10
NUM_CLASSES = 9

# ==============================
# LOAD IMAGE PATHS
# ==============================
image_paths = []
labels = []

for split in ["Train", "Test"]:
    split_path = os.path.join(DATASET_PATH, split)
    for class_name in os.listdir(split_path):
        class_path = os.path.join(split_path, class_name)
        for img in os.listdir(class_path):
            image_paths.append(os.path.join(class_path, img))
            labels.append(class_name)

image_paths = np.array(image_paths)
labels = np.array(labels)

le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

# ==============================
# TF DATA PIPELINE
# ==============================
def load_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = preprocess_input(image)
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

def build_dataset(paths, labels, training=True):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)

    if training:
        ds = ds.map(lambda x, y: (
            tf.image.random_flip_left_right(x),
            y
        ), num_parallel_calls=tf.data.AUTOTUNE)
        ds = ds.shuffle(1024)

    ds = ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return ds

# ==============================
# MODEL BUILDER
# ==============================
def build_model():
    base_model = EfficientNetB3(
        weights="imagenet",
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )

    base_model.trainable = False

    inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

    model = keras.Model(inputs, outputs)

    return model, base_model

# ==============================
# CROSS VALIDATION TRAINING
# ==============================
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracies = []

for fold, (train_idx, val_idx) in enumerate(skf.split(image_paths, labels_encoded)):
    print(f"\n🔥 Training Fold {fold+1}")

    train_ds = build_dataset(image_paths[train_idx], labels_encoded[train_idx], training=True)
    val_ds = build_dataset(image_paths[val_idx], labels_encoded[val_idx], training=False)

    model, base_model = build_model()

    # Compile Head
    model.compile(
        optimizer=keras.optimizers.Adam(1e-4),
        loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
        metrics=["accuracy"]
    )

    callbacks = [
        keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.3)
    ]

    # Train Head
    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS_HEAD,
        callbacks=callbacks,
        verbose=1
    )

    # Fine-tune last 40 layers
    base_model.trainable = True
    for layer in base_model.layers[:-40]:
        layer.trainable = False

    model.compile(
        optimizer=keras.optimizers.Adam(5e-6),
        loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
        metrics=["accuracy"]
    )

    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS_FINE,
        callbacks=callbacks,
        verbose=1
    )

    loss, acc = model.evaluate(val_ds, verbose=0)
    print(f"Fold {fold+1} Accuracy: {acc:.4f}")

    fold_accuracies.append(acc)
    model.save(f"efficientnet_fold_{fold+1}.keras")

# ==============================
# FINAL RESULT
# ==============================
print("\n📊 Fold Accuracies:", fold_accuracies)
print("🔥 Average Accuracy:", np.mean(fold_accuracies))
